In [7]:
import os
import sys
import selenium
from time import sleep
from selenium import webdriver
from bs4 import BeautifulSoup as soups

In [24]:
keyword = input('검색어 : ')
maxImages = int(input('다운로드 시도할 최대 이미지 수 : '))

# 프로젝트에 미리 생성해놓은 crawled_img폴더 안에 하위 폴더 생성
# 폴더명에는 입력한 키워드, 이미지 수 정보를 표시함
path = 'crawled_img/'+keyword+'_'+str(maxImages)

try:
    # 중복되는 폴더 명이 없다면 생성
    if not os.path.exists(path):
        os.makedirs(path)
    # 중복된다면 문구 출력 후 프로그램 종료
    else:
        print('이전에 같은 [검색어, 이미지 수]로 다운로드한 폴더가 존재합니다.')
        sys.exit(0)
except OSError:
    print ('os error')
    sys.exit(0)



pages = int((maxImages-1)/100)+1 #한 페이지당 표시되는 이미지 수(100)을 참고하여 확인할 페이지 수 계산
imgCount = 0 # 추출 시도 이미지 수
success = 0 # 추출 성공 이미지 수
finish = False # 이미지에 모두 접근했는지 여부

# 크롬 드라이버 설정
# (크롤링할 때 웹 페이지 띄우지 않음, gpu 사용 안함, 한글 지원, user-agent 헤더 추가)
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--disable-gpu')
chrome_options.add_argument('lang=ko_KR')
chrome_options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/89.0.4389.82 Safari/537.36")
driver = webdriver.Chrome('C:\chromedriver\chromedriver.exe',chrome_options=chrome_options)

for i in range(1,int(pages)+1):
    #웹 페이지 접근 후 1초동안 로드를 기다림
    driver.get('https://pixabay.com/images/search/'+keyword+'/?pagi='+str(i))
    sleep(1)

    #크롤링이 가능하도록 html코드 가공
    html = driver.page_source
    soup = soups(html,'html.parser')

    imgs = soup.select('div.flex_grid.credits.search_results img') #요소 선택

    #마지막 페이지 여부 결정
    lastPage=False
    if len(imgs)!=100:
        lastPage=True

    #5번 제목에서 설명함
    for img in imgs:
        srcset = ""
        if img.get('srcset')==None:
            srcset = img.get('data-lazy-srcset')
        else: 
            srcset = img.get('srcset')


        src = ""
        if len(srcset):
            src = str(srcset).split()[0] #가장 작은 사이즈의 이미지 경로 추출
            print(src)
            filename = src.split('/')[-1] #이미지 경로에서 날짜 부분뒤의 순 파일명만 추출
            print(filename)
            saveUrl = path+'/'+filename #저장 경로 결정
            print(saveUrl)

            #파일 저장
            #user-agent 헤더를 가지고 있어야 접근 허용하는 사이트도 있을 수 있음(pixabay가 이에 해당)
            req = urllib.request.Request(src, headers={'User-Agent': 'Mozilla/5.0'})
            try:
                imgUrl = urllib.request.urlopen(req).read() #웹 페이지 상의 이미지를 불러옴
                with open(saveUrl,"wb") as f: #디렉토리 오픈
                    f.write(imgUrl) #파일 저장
                success+=1
            except urllib.error.HTTPError:
                print('에러')
                sys.exit(0)

        imgCount+=1

        if imgCount==maxImages:
            finish = True #입력한 이미지 수 만큼 모두 접근했음을 알림
            break
    
    #finish가 참이거나 더 이상 접근할 이미지가 없을 경우 크롤링 종료
    if finish or lastPage:
        break

print('성공 : '+str(success)+', 실패 : '+str(maxImages-success))

C:\Users\gureh\AppData\Local\Temp\ipykernel_27868\3284971777.py:31: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome('C:\chromedriver\chromedriver.exe',chrome_options=chrome_options)
C:\Users\gureh\AppData\Local\Temp\ipykernel_27868\3284971777.py:31: DeprecationWarning: use options instead of chrome_options
  driver = webdriver.Chrome('C:\chromedriver\chromedriver.exe',chrome_options=chrome_options)


성공 : 0, 실패 : 100


In [10]:
import os

In [12]:
os.getcwd()

'c:\\Users\\gureh\\image crwolling'

In [2]:
import threading
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.keys import Keys
import time
import urllib.request
import os

search = "피망"
save_path = "C:/Users/gureh/image crwolling/crawled_img"

# 저장 폴더가 존재하지 않을 경우 생성
if not os.path.exists(save_path):
    os.makedirs(save_path)

service = Service('C:/chrome/chromedriver.exe')
driver = webdriver.Chrome(service=service)

driver.get("https://www.google.co.kr/imghp?hl=ko&tab=wi&authuser=0&ogbl")
elem = driver.find_element("name", "q")  # find_element_by_name 대신 find_element("name", "q") 사용
elem.send_keys(search)
elem.send_keys(Keys.RETURN)

SCROLL_PAUSE_TIME = 3
last_height = driver.execute_script("return document.body.scrollHeight")
SAVE_FLAG = False


def timeout(limit_time):  # timeout
    start = time.time()
    while True:
        if time.time() - start > limit_time or SAVE_FLAG:
            raise Exception('timeout. or image saved.')


while True:
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(SCROLL_PAUSE_TIME)
    new_height = driver.execute_script("return document.body.scrollHeight")
    if new_height == last_height:
        try:
            driver.find_element("css selector", ".mye4qd").click()  # find_element_by_css_selector 대신 find_element("css selector", ".mye4qd") 사용
        except:
            break
    last_height = new_height

images = driver.find_elements("css selector", ".rg_i.Q4LuWd")  # find_elements_by_css_selector 대신 find_elements("css selector", ".rg_i.Q4LuWd") 사용
count = 0
for image in images:
    SAVE_FLAG = False
    timer = threading.Thread(target=timeout, args=(60,))  # 타임아웃 제한 시간을 60초로 변경
    try:
        image.click()
        time.sleep(3)
        # 이미지가 로드될 때까지 대기
        loaded = False
        for _ in range(10):
            if driver.find_elements("xpath", '//*[@id="Sva75c"]/div/div/div[3]/div[2]/c-wiz/div/div[1]/div[1]/div[2]/div[1]/a/img'):
                loaded = True
                break
            time.sleep(1)
        if not loaded:
            raise Exception('Image loading failed.')

        timer.start()
        img_url = driver.find_element("xpath", '//*[@id="Sva75c"]/div/div/div[3]/div[2]/c-wiz/div/div[1]/div[1]/div[2]/div[1]/a/img').get_attribute("src")  # find_element_by_xpath 대신 find_element("xpath", xpath) 사용
        urllib.request.urlretrieve(img_url, os.path.join(save_path, f"{search}_{count:04}.jpg"))
        print('Save images:', os.path.join(save_path, f"{search}_{count:04}.jpg"))
        SAVE_FLAG = True
        count += 1
        if timer.is_alive():
            timer.join()
    except Exception as e:
        if timer.is_alive():
            timer.join()
        pass

print('driver end. Total images:', count)
driver.close()




driver end. Total images: 0


In [6]:
import threading
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time
import urllib.request
import os

# 디렉토리 생성
save_dir = 'C:/Users/gureh/image_crawling/crawled_img/'
os.makedirs(save_dir, exist_ok=True)

search = "피망"
driver = webdriver.Chrome('C:/chrome/chromedriver.exe')
# ... 이하 생략 ...

search = "피망"
driver = webdriver.Chrome('C:/chrome/chromedriver.exe')
driver.get("https://www.google.co.kr/imghp?hl=ko&tab=wi&authuser=0&ogbl")
elem = driver.find_element("name", "q")
elem.send_keys(search)
elem.send_keys(Keys.RETURN)

SCROLL_PAUSE_TIME = 3
last_height = driver.execute_script("return document.body.scrollHeight")

while True:
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(SCROLL_PAUSE_TIME)
    new_height = driver.execute_script("return document.body.scrollHeight")
    if new_height == last_height:
        try:
            driver.find_element_by_css_selector(".mye4qd").click()
        except:
            break
    last_height = new_height

images = driver.find_elements("xpath", '//*[@class="rg_i Q4LuWd"]')
count = 0
for image in images:
    try:
        image_url = image.get_attribute("src")
        urllib.request.urlretrieve(image_url, "C:/Users/gureh/image_crawling/crawled_img/img_{0:04}.jpg".format(count))
        print('Saved image:', "C:/Users/gureh/image_crawling/crawled_img/img_{0:04}.jpg".format(count))
        count += 1
    except Exception as e:
        print('Error occurred while downloading image:', str(e))

print('Total images downloaded:', count)
driver.close()


C:\Users\gureh\AppData\Local\Temp\ipykernel_3108\981421244.py:13: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome('C:/chrome/chromedriver.exe')
C:\Users\gureh\AppData\Local\Temp\ipykernel_3108\981421244.py:17: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome('C:/chrome/chromedriver.exe')


Saved image: C:/Users/gureh/image_crawling/crawled_img/img_0000.jpg
Saved image: C:/Users/gureh/image_crawling/crawled_img/img_0001.jpg
Saved image: C:/Users/gureh/image_crawling/crawled_img/img_0002.jpg
Saved image: C:/Users/gureh/image_crawling/crawled_img/img_0003.jpg
Saved image: C:/Users/gureh/image_crawling/crawled_img/img_0004.jpg
Saved image: C:/Users/gureh/image_crawling/crawled_img/img_0005.jpg
Saved image: C:/Users/gureh/image_crawling/crawled_img/img_0006.jpg
Saved image: C:/Users/gureh/image_crawling/crawled_img/img_0007.jpg
Saved image: C:/Users/gureh/image_crawling/crawled_img/img_0008.jpg
Saved image: C:/Users/gureh/image_crawling/crawled_img/img_0009.jpg
Saved image: C:/Users/gureh/image_crawling/crawled_img/img_0010.jpg
Saved image: C:/Users/gureh/image_crawling/crawled_img/img_0011.jpg
Saved image: C:/Users/gureh/image_crawling/crawled_img/img_0012.jpg
Saved image: C:/Users/gureh/image_crawling/crawled_img/img_0013.jpg
Saved image: C:/Users/gureh/image_crawling/crawl

In [8]:
import threading
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time
import urllib.request
import datetime

def save_images(search, save_dir, max_images):
    driver = webdriver.Chrome('C:/chrome/chromedriver.exe')
    driver.get("https://www.google.co.kr/imghp?hl=ko&tab=wi&authuser=0&ogbl")
    elem = driver.find_element("name", "q")
    elem.send_keys(search)
    elem.send_keys(Keys.RETURN)

    SCROLL_PAUSE_TIME = 3
    last_height = driver.execute_script("return document.body.scrollHeight")

    while True:
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(SCROLL_PAUSE_TIME)
        new_height = driver.execute_script("return document.body.scrollHeight")
        if new_height == last_height:
            try:
                driver.find_element_by_css_selector(".mye4qd").click()
            except:
                break
        last_height = new_height

    images = driver.find_elements("xpath", '//*[@class="rg_i Q4LuWd"]')
    count = 0
    for image in images:
        if count >= max_images:
            break

        try:
            image_url = image.get_attribute("src")
            current_time = datetime.datetime.now().strftime("%Y%m%d%H%M%S")
            file_name = f"img_{current_time}_{count:04}.jpg"
            urllib.request.urlretrieve(image_url, f"{save_dir}/{file_name}")
            print('Saved image:', f"{save_dir}/{file_name}")
            count += 1
        except Exception as e:
            print('Error occurred while downloading image:', str(e))

    print('Total images downloaded:', count)
    driver.close()

search = "피망"
save_dir = "C:/Users/gureh/image_crawling"
max_images = 100

save_images(search, save_dir, max_images)


C:\Users\gureh\AppData\Local\Temp\ipykernel_3108\3080553288.py:9: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome('C:/chrome/chromedriver.exe')


Error occurred while downloading image: [Errno 2] No such file or directory: 'C:/Users/gureh/image_crawling/crawled_img/img_20230603162057_0000.jpg'
Error occurred while downloading image: [Errno 2] No such file or directory: 'C:/Users/gureh/image_crawling/crawled_img/img_20230603162057_0000.jpg'
Error occurred while downloading image: [Errno 2] No such file or directory: 'C:/Users/gureh/image_crawling/crawled_img/img_20230603162057_0000.jpg'
Error occurred while downloading image: [Errno 2] No such file or directory: 'C:/Users/gureh/image_crawling/crawled_img/img_20230603162057_0000.jpg'
Error occurred while downloading image: [Errno 2] No such file or directory: 'C:/Users/gureh/image_crawling/crawled_img/img_20230603162057_0000.jpg'
Error occurred while downloading image: [Errno 2] No such file or directory: 'C:/Users/gureh/image_crawling/crawled_img/img_20230603162057_0000.jpg'
Error occurred while downloading image: [Errno 2] No such file or directory: 'C:/Users/gureh/image_crawlin

KeyboardInterrupt: 

In [2]:
import threading
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time
import urllib.request
import datetime
import os

def save_images(search, save_dir, max_images):
    # 폴더 생성
    folder_name = search.replace(" ", "_")  # 공백을 언더스코어로 대체하여 폴더명 생성
    folder_path = os.path.join(save_dir, folder_name)
    os.makedirs(folder_path, exist_ok=True)

    driver = webdriver.Chrome('C:/chrome/chromedriver.exe')
    driver.get("https://www.google.co.kr/imghp?hl=ko&tab=wi&authuser=0&ogbl")
    elem = driver.find_element("name", "q")
    elem.send_keys(search)
    elem.send_keys(Keys.RETURN)

    SCROLL_PAUSE_TIME = 3
    last_height = driver.execute_script("return document.body.scrollHeight")

    while True:
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(SCROLL_PAUSE_TIME)
        new_height = driver.execute_script("return document.body.scrollHeight")
        if new_height == last_height:
            try:
                driver.find_element_by_css_selector(".mye4qd").click()
            except:
                break
        last_height = new_height

    images = driver.find_elements("xpath", '//*[@class="rg_i Q4LuWd"]')
    count = 0
    for image in images:
        if count >= max_images:
            break

        try:
            image_url = image.get_attribute("src")
            current_time = datetime.datetime.now().strftime("%Y%m%d%H%M%S")
            file_name = f"img_{current_time}_{count:04}.jpg"
            file_path = os.path.join(folder_path, file_name)
            urllib.request.urlretrieve(image_url, file_path)
            print('Saved image:', file_path)
            count += 1
        except Exception as e:
            print('Error occurred while downloading image:', str(e))

    print('Total images downloaded:', count)
    driver.close()

search = "피망"
save_dir = "C:\\Users\\asiae\\chage save file\\crawled_img"
max_images = 100

save_images(search, save_dir, max_images)


C:\Users\asiae\AppData\Local\Temp\ipykernel_6920\238027653.py:15: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome('C:/chrome/chromedriver.exe')


Saved image: C:\Users\asiae\chage save file\crawled_img\피망\img_20230607124919_0000.jpg
Saved image: C:\Users\asiae\chage save file\crawled_img\피망\img_20230607124919_0001.jpg
Saved image: C:\Users\asiae\chage save file\crawled_img\피망\img_20230607124919_0002.jpg
Saved image: C:\Users\asiae\chage save file\crawled_img\피망\img_20230607124919_0003.jpg
Saved image: C:\Users\asiae\chage save file\crawled_img\피망\img_20230607124919_0004.jpg
Saved image: C:\Users\asiae\chage save file\crawled_img\피망\img_20230607124919_0005.jpg
Saved image: C:\Users\asiae\chage save file\crawled_img\피망\img_20230607124919_0006.jpg
Saved image: C:\Users\asiae\chage save file\crawled_img\피망\img_20230607124919_0007.jpg
Saved image: C:\Users\asiae\chage save file\crawled_img\피망\img_20230607124919_0008.jpg
Saved image: C:\Users\asiae\chage save file\crawled_img\피망\img_20230607124919_0009.jpg
Saved image: C:\Users\asiae\chage save file\crawled_img\피망\img_20230607124919_0010.jpg
Saved image: C:\Users\asiae\chage save file